En `pandas` tenemos varias posibilidades para leer datos y similares posibilidades para escribirlos.

# Leamos unos datos de viento

En la carpeta *Datos* tenemos un fichero que se llama *mast.txt* con el siguiente formato:

    130904 0000  2.21  2.58 113.5   999.99 999.99  99.99 9999.99 9999.99  0.11
    130904 0010  1.69  2.31  99.9   999.99 999.99  99.99 9999.99 9999.99  0.35
    130904 0020  1.28  1.50  96.0   999.99 999.99  99.99 9999.99 9999.99  0.08
    130904 0030  1.94  2.39  99.2   999.99 999.99  99.99 9999.99 9999.99  0.26
    130904 0040  2.17  2.67 108.4   999.99 999.99  99.99 9999.99 9999.99  0.23
    130904 0050  2.25  2.89 105.0   999.99 999.99  99.99 9999.99 9999.99  0.35
    ...

Lo podemos leer de la siguiente forma:

In [ ]:
# primero hacemos los imports de turno
import os
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

np.random.seed(19760812)
%matplotlib inline

In [ ]:
ipath = os.path.join('Datos', 'mast.txt')
wind = pd.read_csv(ipath)
wind.head(3)

In [ ]:
wind = pd.read_csv(ipath, sep = "\s*")
# Cuando trabajamos con texto separado por espacios podemos usar la keyword delim_whitespace:
# wind = pd.read_csv(path, delim_whitespace = True)
wind.head(3)

In [ ]:
cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols)
wind.head(3)

In [ ]:
cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols,
                   parse_dates = [[0, 1]])
wind.head(3)

<div class="alert alert-danger">
<p>Dependiendo de tu sistema operativo puede que las fechas sean las correctas o no. Ahora no te preocupes de ellos. Más adelante lidiaremos con ello</p>
</div>

In [ ]:
cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols,
                   parse_dates = [[0, 1]], index_col = 0)
wind.head(3)

In [ ]:
cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols,
                   parse_dates = {'timestamp': [0, 1]}, index_col = 0)
wind.head(3)

In [ ]:
# The previous code is equivalent to
cols = ['Date', 'time', 'wspd', 'wspd_max', 'wdir',
        'x1', 'x2', 'x3', 'x4', 'x5', 
        'wspd_std']
wind = pd.read_csv(ipath, sep = "\s*", names = cols,
                   parse_dates = [[0, 1]], index_col = 0)
wind.index.name = 'Timestamp'
wind.head(3)

In [ ]:
# En la anterior celda de código podemos cambiar los 0's y 1's de 
# parse_dates e index_col por los nombres de las columnas
# Probadlo!!!


In [ ]:
help(pd.read_csv)

Con unas pocas líneas de código hemos conseguido leer un fichero de datos separado por espacios, hemos conseguido leer dos columnas y transformarlas a fechas (de forma mágica), hemos conseguido indicar que esas fechas se consideren el índice (solo puede haber un registro en cada momento),...

# ¡¡Warning!! índices repetidos

<br>
<div class="alert alert-danger">
<h3>Nota:</h3>
<p>Nada impide tener dos índices repetidos. Tened cuidado con esto ya que puede ser una fuente de errores.</p>
</div>

In [ ]:
tmp = pd.DataFrame([1,10,100, 1000], index = [1,1,2,2], columns = ['values'])

In [ ]:
tmp

In [ ]:
print(tmp['values'][1], tmp['values'][2], sep = '\n')

# ¡¡Warning!! cuando hagamos conversión de fechas desde *strings*

<br>
<div class="alert alert-danger">
<h3>Nota:</h3>
<p>Si dejáis que pandas *parsee* las fechas escribid tests para ello pues puede haber errores en la conversión <b>automágica</b>.</p>
</div>

In [ ]:
# Ejemplo de error en fechas:

index = [
    '01/01/2015 00:00',
    '02/01/2015 00:00',
    '03/01/2015 00:00',
    '04/01/2015 00:00',
    '05/01/2015 00:00',
    '06/01/2015 00:00',
    '07/01/2015 00:00',
    '08/01/2015 00:00',
    '09/01/2015 00:00',
    '10/01/2015 00:00',
    '11/01/2015 00:00',
    '12/01/2015 00:00',
    '13/01/2015 00:00',
    '14/01/2015 00:00',
    '15/01/2015 00:00'
]
values = np.random.randn(len(index))

tmp = pd.DataFrame(values, index = pd.to_datetime(index), columns = ['col1'])

In [ ]:
display(tmp)
tmp.plot.line(figsize = (12, 6))

Para evitar lo anterior podemos crear nuestro propio parser de fechas a, por ejemplo, `pd.read_csv`:

In [ ]:
import datetime as dt
import io

def dateparser(date):
    date, time = date.split()
    DD, MM, YY = date.split('/')
    hh, mm = time.split(':')
    return dt.datetime(int(YY), int(MM), int(DD), int(hh), int(mm))

virtual_file = io.StringIO("""01/01/2015 00:00, 1
02/01/2015 00:00, 2
03/01/2015 00:00, 3
04/01/2015 00:00, 4
05/01/2015 00:00, 5
06/01/2015 00:00, 6
07/01/2015 00:00, 7
08/01/2015 00:00, 8
09/01/2015 00:00, 9
10/01/2015 00:00, 10
11/01/2015 00:00, 11
12/01/2015 00:00, 12
13/01/2015 00:00, 13
14/01/2015 00:00, 14
15/01/2015 00:00, 15
""")

tmp_wrong = pd.read_csv(virtual_file, parse_dates = [0], index_col = 0, names = ['Date', 'values'])

virtual_file = io.StringIO("""01/01/2015 00:00, 1
02/01/2015 00:00, 2
03/01/2015 00:00, 3
04/01/2015 00:00, 4
05/01/2015 00:00, 5
06/01/2015 00:00, 6
07/01/2015 00:00, 7
08/01/2015 00:00, 8
09/01/2015 00:00, 9
10/01/2015 00:00, 10
11/01/2015 00:00, 11
12/01/2015 00:00, 12
13/01/2015 00:00, 13
14/01/2015 00:00, 14
15/01/2015 00:00, 15
""")

tmp_right = pd.read_csv(virtual_file, parse_dates = True, index_col = 0, names = ['Date', 'values'],
                        date_parser = dateparser)

In [ ]:
display(tmp_wrong)
display(tmp_right)

# Vamos a salvar el resultado en formato csv

In [ ]:
opath = os.path.join('Datos', 'mast_2.csv')
#wind.to_csv(opath)
wind.iloc[0:100].to_csv(opath)

# ... o en formato json

In [ ]:
#wind.to_json(opath.replace('csv', 'json'))
wind.iloc[0:100].to_json(opath.replace('csv', 'json'))

# ... o en una tabla HTML

In [ ]:
# Si son muchos datos no os lo recomiendo, es lento
#wind.to_html(opath.replace('csv', 'html'))
wind.iloc[0:100].to_html(opath.replace('csv', 'html'))

# ... o en formato xlsx

Seguramente debáis instalar `xlsxwriter`, `openpyxl`, `wlrd/xlwt`,...

In [ ]:
writer = pd.ExcelWriter(opath.replace('csv', 'xlsx'))
#wind.to_excel(writer, sheet_name= "Mi hoja 1")
wind.iloc[0:100].to_excel(writer, sheet_name= "Mi hoja 1")
writer.save()

In [ ]:
# Ahora que tenemos los ficheros en formato json, html, xlsx,..., podéis practicar a abrirlos con las
# funciones pd.read_*
